# Cleaning Data: Homework

In class, we discussed a few different ways to clean data — simple Python methods, Pandas versions of those methods, and regular expressions (both in plain Python and in Pandas). You are welcome to use whatever combination of those approaches you'd like. If you really want to stretch your skills, try solving the questions a few different ways.

## 0) Load `pets.csv` (provided in CourseWorks)

... and assign the data to a variable named `pets`.

In [52]:
import pandas as pd
import re

# Read the data from the CSV file
pets = pd.read_csv('pets.csv')

# Print the first few rows of the data
pets.head(10)


,name,species,weight,years_old
0,"smith, fido",dog,5 lbs,1
1,"SMITH,, Bella",CAT,7 lbs,10
2,"Ortiz, Milo",dog,15 lbs,5
3,"Smith, Goldie",goldfish,7 oz,???
4,"Kim, Fuzz",cat,3 lbs,5 (five)
5,"Jones, Las Vegas",DOG,12 lbs,unknown
6,Jones; fifi,puppy,2 lbs,0.25
7,"Lee, nemo",gold fish,10 oz,2
8,"Chan, ZIGGY",kitten,2 lbs,0.5
9,"Chan, Zog",dog,20 lbs,7


We're going to clean the data so that you can __programmatically__ answer the following questions:

- Which family owns the most pets? (All pets have been given the surname of the family that owns them. Assume that there's only one family with each surname.)
---
- How many of each kind of animal are there? (Puppies should count as dogs, and kittens as cats.)
---
- What is the total weight of the pets? (And which family's pets weigh the most, total?)
---
- Of the *known* ages, what's the average pet age?

## 1) Create a new column in the DataFrame called `last_name`, containing *just* the pet's surname

... normalized so that `.value_counts()` will count `smith, fido` and `SMITH, Bella` as being from the same family.

In [53]:
# Create a new column in the DataFrame called `last_name`, containing *just* the pet's surname

# Split the `name` column on ,space or ;space or,,space character and take the first element from the resulting list
# write the result to the `last_name` column and use .title() to capitalize the first letter of the last name
pets['last_name'] = pets['name'].str.split(r'[,;]\s*').str[0].str.title()
pets.head(30)


,name,species,weight,years_old,last_name
0,"smith, fido",dog,5 lbs,1,Smith
1,"SMITH,, Bella",CAT,7 lbs,10,Smith
2,"Ortiz, Milo",dog,15 lbs,5,Ortiz
3,"Smith, Goldie",goldfish,7 oz,???,Smith
4,"Kim, Fuzz",cat,3 lbs,5 (five),Kim
5,"Jones, Las Vegas",DOG,12 lbs,unknown,Jones
6,Jones; fifi,puppy,2 lbs,0.25,Jones
7,"Lee, nemo",gold fish,10 oz,2,Lee
8,"Chan, ZIGGY",kitten,2 lbs,0.5,Chan
9,"Chan, Zog",dog,20 lbs,7,Chan


## 2) Check how many distinct last names you see

(There should be 6.)

In [54]:
# count the distinct values in the `last_name` column
pets['last_name'].nunique()

6

## 3) Calculate which family owns the most pets

(All pets have been given the surname of the family that owns them. Assume that there's only one family with each surname.)

In [55]:
# Calculate which family owns the most pets
pets['last_name'].value_counts().head(1)

last_name
Smith    3
Name: count, dtype: int64

## 4) Create a new column called `animal_type`, which standardizes the `species` column

(Puppies should count as dogs, and kittens as cats; and there should only be one spelling of `gold fish`/`goldfish`.)

In [62]:
# Create a new column in the DataFrame called `animal_type`, replace everthing have ..pupp. between with dog and kitten with cat
# and write gold fish to goldfish
pets["animal_type"] = (
    pets["species"]
    .str.replace(r".*pupp.*", "dog", regex=True)
    .str.replace(r".*kitten.*", "cat", regex=True)
    .str.replace("gold fish", "goldfish")
    .str.lower()
)
pets["animal_type"]

0         dog
1         cat
2         dog
3    goldfish
4         cat
5         dog
6         dog
7    goldfish
8         cat
9         dog
Name: animal_type, dtype: object

## 5) How many of each kind of animal are there?

In [63]:
# How many of each kind of animal are there?
pets["animal_type"].value_counts()

animal_type
dog         5
cat         3
goldfish    2
Name: count, dtype: int64

## 6) Create a new column called `weight_quantity`, which represents the numeric component of the `weight` column, as an integer

(i.e., `"10 oz"` -> `10`)

In [65]:
pets['weight_quantity'] = pets['weight'].str[:2].astype(float)
pets

,name,species,weight,years_old,last_name,animal_type,weight_quantity
0,"smith, fido",dog,5 lbs,1,Smith,dog,5.0
1,"SMITH,, Bella",CAT,7 lbs,10,Smith,cat,7.0
2,"Ortiz, Milo",dog,15 lbs,5,Ortiz,dog,15.0
3,"Smith, Goldie",goldfish,7 oz,???,Smith,goldfish,7.0
4,"Kim, Fuzz",cat,3 lbs,5 (five),Kim,cat,3.0
5,"Jones, Las Vegas",DOG,12 lbs,unknown,Jones,dog,12.0
6,Jones; fifi,puppy,2 lbs,0.25,Jones,dog,2.0
7,"Lee, nemo",gold fish,10 oz,2,Lee,goldfish,10.0
8,"Chan, ZIGGY",kitten,2 lbs,0.5,Chan,cat,2.0
9,"Chan, Zog",dog,20 lbs,7,Chan,dog,20.0


## 7) Create a new column called `weight_unit`, which represents the units component of the `weight` column

(i.e., `"10 oz"` -> `"oz"`)

In [67]:
pets['weight_unit'] = pets['weight'].str.extract(r' (.*)')
pets

,name,species,weight,years_old,last_name,animal_type,weight_quantity,weight_unit
0,"smith, fido",dog,5 lbs,1,Smith,dog,5.0,lbs
1,"SMITH,, Bella",CAT,7 lbs,10,Smith,cat,7.0,lbs
2,"Ortiz, Milo",dog,15 lbs,5,Ortiz,dog,15.0,lbs
3,"Smith, Goldie",goldfish,7 oz,???,Smith,goldfish,7.0,oz
4,"Kim, Fuzz",cat,3 lbs,5 (five),Kim,cat,3.0,lbs
5,"Jones, Las Vegas",DOG,12 lbs,unknown,Jones,dog,12.0,lbs
6,Jones; fifi,puppy,2 lbs,0.25,Jones,dog,2.0,lbs
7,"Lee, nemo",gold fish,10 oz,2,Lee,goldfish,10.0,oz
8,"Chan, ZIGGY",kitten,2 lbs,0.5,Chan,cat,2.0,lbs
9,"Chan, Zog",dog,20 lbs,7,Chan,dog,20.0,lbs


## 8) Create a new column called `unit_factor`, which should equal `1` when `weight_unit` is `"lbs"` and `0.0625` when `weight_unit` is `"oz"`

(i.e., `"10 oz"` -> `"oz"`)

You could do this by defining a function and using `column.apply(my_function)` or through Pandas' `.replace({ ... })` method, which we didn't cover in class but is somewhat easier here. That would look like this (yes, I'm giving you the answer here):

```python
pets["unit_factor"] = (
    pets["weight_unit"]
    .replace({
        "lbs": 1,
        "oz": 0.0625
    })
)
```

In [86]:
pets["unit_factor"] = (
    pets["weight_unit"]
    .replace({
        "lbs": 1,
        "oz": 0.0625
    })
)

/var/folders/fg/9vmq_tp10lv_kw_hsjpvgq840000gn/T/ipykernel_56405/205097127.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({"lbs": 1, "oz": 0.0625})


In [69]:
# Create a new column called `unit_factor`, which should equal `1` when `weight_unit` is `"lbs"` and `0.0625` when `weight_unit` is `"oz"`
pets['unit_factor'] = pets['weight_unit'].map({'lbs': 1, 'oz': 0.0625})
pets

,name,species,weight,years_old,last_name,animal_type,weight_quantity,weight_unit,unit_factor
0,"smith, fido",dog,5 lbs,1,Smith,dog,5.0,lbs,1.0000
1,"SMITH,, Bella",CAT,7 lbs,10,Smith,cat,7.0,lbs,1.0000
2,"Ortiz, Milo",dog,15 lbs,5,Ortiz,dog,15.0,lbs,1.0000
3,"Smith, Goldie",goldfish,7 oz,???,Smith,goldfish,7.0,oz,0.0625
4,"Kim, Fuzz",cat,3 lbs,5 (five),Kim,cat,3.0,lbs,1.0000
5,"Jones, Las Vegas",DOG,12 lbs,unknown,Jones,dog,12.0,lbs,1.0000
6,Jones; fifi,puppy,2 lbs,0.25,Jones,dog,2.0,lbs,1.0000
7,"Lee, nemo",gold fish,10 oz,2,Lee,goldfish,10.0,oz,0.0625
8,"Chan, ZIGGY",kitten,2 lbs,0.5,Chan,cat,2.0,lbs,1.0000
9,"Chan, Zog",dog,20 lbs,7,Chan,dog,20.0,lbs,1.0000


## 8) Create a new column called `weight_lbs`, which calculates the pet's weight in lbs by multiplying `weight_quantity` by `unit_factor`

In [72]:
pets['weight_lbs'] = pets['weight_quantity'] * pets['unit_factor']
pets

,name,species,weight,years_old,last_name,animal_type,weight_quantity,weight_unit,unit_factor,weight_lbs
0,"smith, fido",dog,5 lbs,1,Smith,dog,5.0,lbs,1.0000,5.0000
1,"SMITH,, Bella",CAT,7 lbs,10,Smith,cat,7.0,lbs,1.0000,7.0000
2,"Ortiz, Milo",dog,15 lbs,5,Ortiz,dog,15.0,lbs,1.0000,15.0000
3,"Smith, Goldie",goldfish,7 oz,???,Smith,goldfish,7.0,oz,0.0625,0.4375
4,"Kim, Fuzz",cat,3 lbs,5 (five),Kim,cat,3.0,lbs,1.0000,3.0000
5,"Jones, Las Vegas",DOG,12 lbs,unknown,Jones,dog,12.0,lbs,1.0000,12.0000
6,Jones; fifi,puppy,2 lbs,0.25,Jones,dog,2.0,lbs,1.0000,2.0000
7,"Lee, nemo",gold fish,10 oz,2,Lee,goldfish,10.0,oz,0.0625,0.6250
8,"Chan, ZIGGY",kitten,2 lbs,0.5,Chan,cat,2.0,lbs,1.0000,2.0000
9,"Chan, Zog",dog,20 lbs,7,Chan,dog,20.0,lbs,1.0000,20.0000


## 9) What is the total weight of the pets?

In [77]:
sum(pets['weight_lbs'])

67.0625

## 10)  And which family's pets weigh the most, total?

In [75]:
# Step 1 & 2: Group by 'family_name' and sum the 'weight'
total_weights_by_family = pets.groupby('last_name')['weight_lbs'].sum()

# Step 3: Find the family with the highest total weight
heaviest_family = total_weights_by_family.idxmax()

# Display the result
print(f"The family with the heaviest pets is: {heaviest_family}")

The family with the heaviest pets is: Chan


## 11) Of the known ages, what's the average pet age?

(You should get `3.84375`; if not, make sure you're accounting for numbers like `0.5` correctly.)

In [85]:
# average pet age. just the number ignore strings
# Extract the age as a number
pets["age"] = pets["years_old"].str.extract(r"(\d+\.?\d*)").astype(float)
pets["age"].mean()

3.84375

---

---

---